In [1]:
import numpy as np
import pandas as pd
import json
import os

In [2]:
def read_json_files(directory):
    files = os.listdir(directory)
    json_files = [file for file in files if file.endswith('.json')]

    dfs = []
    for file in json_files:
        file_path = os.path.join(directory, file)
        json_data = pd.read_json(file_path)
        dfs.append(json_data)

    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

df = read_json_files('.')

df

,image,objects
0,"{'fileName': '20220622_SD04_7973.JPG', 'detail...","[{'bndbox': {'minX': 0.904863, 'minY': 0.90394..."
1,"{'fileName': '20220622_SD04_7892.JPG', 'detail...","[{'bndbox': {'minX': 0.560254, 'minY': 0.41549..."
2,"{'fileName': '20220622_SD04_7779.JPG', 'detail...","[{'bndbox': {'minX': 0.48625799999999997, 'min..."
3,"{'fileName': '20220622_SD04_7965.JPG', 'detail...","[{'bndbox': {'minX': 0.293869, 'minY': 0.54554..."
4,"{'fileName': '20220622_SD04_7904.JPG', 'detail...","[{'bndbox': {'minX': 0.47357299999999997, 'min..."
...,...,...
6746,"{'fileName': '20220627_SD04_8847.JPG', 'detail...","[{'bndbox': {'minX': 0.21301799999999999, 'min..."
6747,"{'fileName': '20220627_SD04_8804.JPG', 'detail...","[{'bndbox': {'minX': 0.5897439999999999, 'minY..."
6748,"{'fileName': '20220627_SD04_8595.JPG', 'detail...","[{'bndbox': {'minX': 0.545652, 'minY': 0.20220..."
6749,"{'fileName': '20220627_SD04_8768.JPG', 'detail...","[{'bndbox': {'minX': 0.268245, 'minY': 0.04438..."


In [3]:
image = df['image'].to_dict()
file_n = []
for i in range(0, len(image)):
    file_n.append(image[i]['fileName'])
    
file_orientation =[]
for i in range(0, len(image)):
    if image[i]['details']['width'] < image[i]['details']['height']:
        orientation_s = 'portait'
        file_orientation.append(orientation_s)
    else: 
        orientation_s = 'landscape'
        file_orientation.append(orientation_s)

jpg_df = pd.DataFrame({'file_name': file_n, 'orientation': file_orientation})
jpg_df = jpg_df.sort_values('file_name')
ind = jpg_df.index
jpg_df = jpg_df.reset_index(drop=True)
jpg_df = jpg_df.reset_index().rename(columns={'index': 'jpg_id'})
jpg_df

,jpg_id,file_name,orientation
0,0,20220308_SD02_2923.JPG,portait
1,1,20220308_SD02_2924.JPG,portait
2,2,20220308_SD02_2946.JPG,portait
3,3,20220308_SD02_2954.JPG,landscape
4,4,20220308_SD02_2955.JPG,landscape
...,...,...,...
6746,6746,20220825_SD05_6957.JPG,portait
6747,6747,20220825_SD05_6958.JPG,portait
6748,6748,20220825_SD05_6960.JPG,portait
6749,6749,20220825_SD05_6963.JPG,portait


In [4]:
ind

Int64Index([5801, 5800, 2462, 2463, 2450, 2456, 2479, 2471, 2473, 2476,
            ...
             109,  116,  128,  112,  120,  104,  108,  117,  127,  114],
           dtype='int64', length=6751)

In [5]:
obj = df['objects'].to_dict()
obj
obj[0][0]['bndbox']
obj[0][0]['category']['name']

'Toy-Grenade'

In [6]:
coord_list = []
bndbox_name_list = []
for i in range(len(obj)):
    coord = []
    bndbox_name = []
    for j in range(len(obj[i])):
        category_name = obj[i][j]['category']['name']
        bndbox_dict = obj[i][j]['bndbox']
        bndbox_tuple = (bndbox_dict['minX'], bndbox_dict['minY'], bndbox_dict['maxX'], bndbox_dict['maxY'])
        
        coord.append(bndbox_tuple)
        bndbox_name.append(category_name)
    coord_list.append(coord)
    bndbox_name_list.append(bndbox_name)

bndbox_df = pd.DataFrame({'bndbox_name': bndbox_name_list, 'coordinates': coord_list})
bndbox_df = bndbox_df.reindex(ind)
bndbox_df = bndbox_df.reset_index(drop=True)
bndbox_df = bndbox_df.reset_index().rename(columns={'index': 'jpg_id'})
bndbox_df.head()

,jpg_id,bndbox_name,coordinates
0,0,"[DF2I_Face, DF2I_Body]","[(0.36186799999999997, 0.0028209999999999997, ..."
1,1,"[DF2I_Face, DF2I_Body]","[(0.34282199999999996, 0.0, 0.8845649999999999..."
2,2,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.152365, 0.35543, 0.5100009999999999, 0.558..."
3,3,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.42407799999999995, 0.587402, 0.61605199999..."
4,4,"[151J_Face, 151J_Body, 110E_Face, 110E_Body, 7...","[(0.47397000000000006, 0.564622, 0.681128, 0.8..."


In [7]:
bndbox_df['coordinates'][0]

[(0.36186799999999997, 0.0028209999999999997, 0.916308, 0.38222799999999996),
 (0.0, 0.0, 0.954399, 1.0)]

In [8]:
coord_df = pd.concat([jpg_df['file_name'], bndbox_df], axis = 1)
coord_df.head(3)
coord_df = coord_df.explode(['bndbox_name', 'coordinates'])
coord_df = coord_df.sort_values('file_name')
coord_df.head(15)

,file_name,jpg_id,bndbox_name,coordinates
0,20220308_SD02_2923.JPG,0,DF2I_Face,"(0.36186799999999997, 0.0028209999999999997, 0..."
0,20220308_SD02_2923.JPG,0,DF2I_Body,"(0.0, 0.0, 0.954399, 1.0)"
1,20220308_SD02_2924.JPG,1,DF2I_Face,"(0.34282199999999996, 0.0, 0.8845649999999999,..."
1,20220308_SD02_2924.JPG,1,DF2I_Body,"(0.0, 0.0028209999999999997, 0.975560999999999..."
2,20220308_SD02_2946.JPG,2,7124_Body,"(0.002116, 0.0, 0.387262, 0.7334269999999999)"
2,20220308_SD02_2946.JPG,2,110E_Body,"(0.101577, 0.09591000000000001, 0.971329, 1.0)"
2,20220308_SD02_2946.JPG,2,7124_Face,"(0.0, 0.0, 0.378797, 0.31029599999999996)"
2,20220308_SD02_2946.JPG,2,151J_Body,"(0.126971, 0.35543, 0.8591709999999999, 0.943583)"
2,20220308_SD02_2946.JPG,2,151J_Face,"(0.152365, 0.35543, 0.5100009999999999, 0.558533)"
2,20220308_SD02_2946.JPG,2,110E_Face,"(0.118506, 0.094499, 0.5100009999999999, 0.389..."


In [9]:
toy_coord = coord_df[coord_df['bndbox_name'].str.contains('toy', case=False)]
monkey_coord = coord_df[~coord_df['bndbox_name'].str.contains('toy', case=False)]

# Toy Coord

In [10]:
pattern = r'Toy-|Toy_|toy_|toy-'
toy_coord = toy_coord[~toy_coord['bndbox_name'].str.contains('triangle|swing', case=False)]
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace(pattern, '')
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Blue Frisbees', 'Blue Frisbee')
toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace('Lucky Clover chew', 'Lucky Clover Chew')

toy_coord.bndbox_name.unique()

/var/folders/7y/b5rc1dg90wjbpqjnn9qnxvf40000gn/T/ipykernel_53182/4288504467.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  toy_coord['bndbox_name'] = toy_coord['bndbox_name'].str.replace(pattern, '')


array(['Unknown', 'Dental Star', 'Dumbbell', 'Grenade', 'Kong',
       'Lucky Clover Chew', 'Megalast Ball', 'Ziggs', 'Blue Frisbee',
       'Dental Kong', 'Zyro', 'Hex', 'S-shaped Toy', 'Challenger Ball'],
      dtype=object)

In [11]:
toy_coord = toy_coord.reset_index(drop=True)
toy_coord = toy_coord.reset_index().rename(columns={'index': 'toy_id'})
toy_coord = toy_coord.rename(columns={'bndbox_name': 'toy'})
toy_coord

,toy_id,file_name,jpg_id,toy,coordinates
0,0,20220308_SD02_3022.JPG,34,Unknown,"(0.0, 0.895628, 0.16083, 1.0)"
1,1,20220308_SD02_3029.JPG,38,Unknown,"(0.7477809999999999, 0.818182, 1.0, 0.97861000..."
2,2,20220317_SD02_3125.JPG,60,Dental Star,"(0.6565219999999999, 0.727354, 0.947826, 0.825..."
3,3,20220317_SD02_3165.JPG,80,Dumbbell,"(0.321212, 0.748602, 1.0, 1.0)"
4,4,20220331_SD02_3342.JPG,163,Grenade,"(0.40405599999999997, 0.365112, 0.528861, 0.61..."
...,...,...,...,...,...
1002,1002,20220810_SD05_6742.JPG,6653,Challenger Ball,"(0.5826089999999999, 0.0, 1.0, 0.3202100000000..."
1003,1003,20220810_SD05_6743.JPG,6654,Challenger Ball,"(0.6869569999999999, 0.001449, 1.0, 0.22458199..."
1004,1004,20220825_SD05_6824.JPG,6695,Kong,"(0.8576849999999999, 0.8910309999999999, 0.944..."
1005,1005,20220825_SD05_6825.JPG,6696,Kong,"(0.8842500000000001, 0.928039, 0.963947, 1.0)"


# Monkey Coord

In [12]:
monkey_coord = monkey_coord[~monkey_coord['bndbox_name'].str.contains('triangle|swing', case=False)]
# Define the dictionary of replacements
replacements = {'Unknown 1': 'Unknown1', 'Unknown 2': 'Unknown2', 'DF21' : 'DF2I', 'OEX': '0EX', '0F1': '0FL'}
# Replace the patterns in the 'bndbox_name' column using the dictionary of replacements
monkey_coord['bndbox_name'] = monkey_coord['bndbox_name'].replace(replacements, regex=True)

In [13]:
monkey_coord.bndbox_name.unique()

array(['DF2I_Face', 'DF2I_Body', '7124_Body', '110E_Body', '7124_Face',
       '151J_Body', '151J_Face', '110E_Face', '67G_Face', '67G_Body',
       '94B_Body', '94B_Face', '72X_Face', '72X_Body', '143H_Body',
       '143H_Face', '69X_Face', '69X_Body', '87J_Body', '87J_Face',
       '14F_Face', '14F_Body', '19J_Face', '19J_Body', '101G_Face',
       'G701_Face', 'G701_Body', '68F_Body', '68F_Face', 'Unknown Body',
       '151J Body', '151J Face', '110E Body', '110E Face', '7124 Face',
       '7124 Body', '49Y Face', '49Y Body', 'Unknown Face', '79G Face',
       '79G Body', '68E Face', '68E Body', '167I Face', '167I Body',
       '86I Face', '86I Body', '59E Face', '59E Body', '134J Body',
       '134J  Face', '35Y Body', '35Y Face', 'G942 Face', 'G942 Body',
       '144H Body', '144H Face', '58I Face', '58I Body', '46J Body',
       '46J Face', '26J Body', '26J Face', 'DF2I Face', 'DF2I Body',
       '48Z Face', '48Z Body', '70G Body', '70G Face', '101G_Body',
       'Unknown1_Body',

In [14]:
monkey_coord[['monkey_name', 'body_part']] = monkey_coord['bndbox_name'].str.split('_| ', 1, expand=True)
monkey_coord = monkey_coord.drop('bndbox_name',axis=1)

In [15]:
monkey_coord.monkey_name.unique()

array(['DF2I', '7124', '110E', '151J', '67G', '94B', '72X', '143H', '69X',
       '87J', '14F', '19J', '101G', 'G701', '68F', 'Unknown', '49Y',
       '79G', '68E', '167I', '86I', '59E', '134J', '35Y', 'G942', '144H',
       '58I', '46J', '26J', '48Z', '70G', 'Unknown1', '114J', '42Z',
       '36J', '0EX', '0FL', '42K', '37I', '68Y', '81G', 'Unknown2', '40J',
       '09X', '58K', '59K', '67K', '129K', '98K', '120K', '126K', '139K',
       '122K', '170K'], dtype=object)

In [16]:
monkey_coord['body_part'] = monkey_coord['body_part'].str.replace(' ', '')
monkey_coord['body_part'] = monkey_coord['body_part'].str.capitalize()
monkey_coord.body_part.unique()

array(['Face', 'Body', None], dtype=object)

In [17]:
monkey_coord = monkey_coord.drop('file_name',axis=1)
monkey_coord.tail(5)

,jpg_id,coordinates,monkey_name,body_part
6748,6748,"(0.0, 0.173975, 0.8955559999999999, 0.876945)",G942,Body
6749,6749,"(0.581475, 0.164074, 0.846746, 0.424328)",G942,Face
6749,6749,"(0.0021219999999999998, 0.168317, 1.0, 0.929279)",G942,Body
6750,6750,"(0.22919399999999998, 0.011314999999999999, 0....",G942,Face
6750,6750,"(0.0042439999999999995, 0.015559, 0.965588, 1.0)",G942,Body


In [33]:
duplicates = monkey_coord[monkey_coord[monkey_coord['monkey_name']=='Unknown']].duplicated('jpg_id', keep=False)]

#monkey_coord[monkey_coord['monkey_name']=='Unknown']['jpg_id'].duplicated(keep=False)
duplicates['jpg_id']
# Count the occurrences of each value in the duplicates
value_counts = duplicates['jpg_id'].value_counts()

# Print the values that occur more than once
more_than_duplicates = value_counts[value_counts > 1]
#print(more_than_duplicates)

SyntaxError: unmatched ']' (703299073.py, line 1)

In [ ]:
monkey_coord[monkey_coord['monkey_name']=='Unknown']['jpg_id'].duplicated(keep=False)

In [31]:
ind = monkey_coord[monkey_coord['monkey_name']=='Unknown']['jpg_id'].duplicated(keep=False)


/var/folders/7y/b5rc1dg90wjbpqjnn9qnxvf40000gn/T/ipykernel_53182/3491308405.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(monkey_coord[ind])


ValueError: cannot reindex from a duplicate axis

In [22]:
 monkey_coord[monkey_coord['monkey_name']=='Unknown']['jpg_id'].duplicated(keep=False)

# monkey_coord[monkey_coord.body_part.isnull()]

46      False
50       True
50       True
60       True
60       True
        ...  
6740     True
6741     True
6741     True
6742     True
6742     True
Name: jpg_id, Length: 1014, dtype: bool

In [19]:
jpg_df.to_csv('../fromjson_photos.csv', index=False)
monkey_coord.to_csv('../fromjson_mCoord.csv', index=False)
toy_coord.to_csv('../fromjson_toyCoord.csv', index = False)